# Community Detection: Amazon Product Co-Purchasing Network

This notebook performs community detection on the Amazon co-purchasing network using multiple algorithms and evaluates them against ground-truth communities.

## Objectives:
1. Load preprocessed graph and ground-truth communities
2. Run all 3 community detection algorithms (Louvain, Label Propagation, Greedy Modularity)
3. Compare execution times and performance
4. Evaluate against ground truth using NMI and ARI
5. Visualize results and analyze which algorithm performs best
6. Generate insights and interpretations


## 1. Import Libraries and Setup

Import all necessary libraries for community detection, evaluation, and visualization.


In [ ]:
# Standard library imports
import os
import sys
import time
from pathlib import Path

# Add src directory to path for imports
if Path.cwd().name == 'notebooks':
    project_root = Path.cwd().parent
else:
    project_root = Path.cwd()

sys.path.insert(0, str(project_root / 'src'))
os.chdir(project_root)
print(f"Project root: {project_root}")
print(f"Working directory: {os.getcwd()}")

# Data manipulation
import pandas as pd
import numpy as np

# Network analysis
import networkx as nx

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Project modules
from data_loader import load_saved_graph, load_communities
from community_detection import (
    run_all_community_detection,
    get_community_sizes,
    communities_to_dict
)
from community_evaluation import (
    load_ground_truth_communities,
    evaluate_communities,
    compare_all_methods,
    analyze_community_overlap
)
from community_visualization import (
    plot_community_size_distribution,
    plot_modularity_comparison,
    plot_evaluation_metrics,
    plot_largest_communities,
    create_community_report
)

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 50)

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid' if 'seaborn-v0_8-darkgrid' in plt.style.available 
              else 'seaborn-darkgrid' if 'seaborn-darkgrid' in plt.style.available 
              else 'ggplot')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")


## 2. Load Preprocessed Graph and Ground-Truth Communities

Load the cleaned graph and ground-truth communities for evaluation.


In [ ]:
# Load the preprocessed graph
graph_path = "data/processed/amazon_graph_cleaned.pkl"
print(f"Loading graph from: {graph_path}")
G = load_saved_graph(graph_path)

print(f"\n✅ Graph loaded successfully!")
print(f"   Nodes: {G.number_of_nodes():,}")
print(f"   Edges: {G.number_of_edges():,}")
print(f"   Type: {'Undirected' if not G.is_directed() else 'Directed'}")

# Load ground-truth communities
communities_path = "data/raw/com-amazon.all.cmty.txt.gz"
print(f"\nLoading ground-truth communities from: {communities_path}")
ground_truth_dict = load_ground_truth_communities(communities_path)

print(f"\n✅ Ground-truth communities loaded!")
print(f"   Number of communities: {len(set(ground_truth_dict.values())):,}")
print(f"   Number of nodes: {len(ground_truth_dict):,}")
print(f"   Coverage: {len(ground_truth_dict) / G.number_of_nodes() * 100:.2f}% of graph nodes")


## 3. Run All Community Detection Algorithms

Run all three community detection algorithms (Louvain, Label Propagation, Greedy Modularity) and time each one.


In [ ]:
# Run all community detection algorithms
print("Running all community detection algorithms...")
print("=" * 60)
print("Note: This may take several minutes for large graphs")
print("=" * 60)

start_time = time.time()
detection_results = run_all_community_detection(G, louvain_resolution=1.0, seed=42)
total_time = time.time() - start_time

print("=" * 60)
print(f"✅ All algorithms completed in {total_time:.2f} seconds ({total_time/60:.2f} minutes)")
print("=" * 60)


## 4. Display Execution Times

Compare the execution time for each algorithm to understand computational costs.


In [ ]:
# Display execution time comparison
if 'timing' in detection_results:
    timing_df = pd.DataFrame([
        {'Algorithm': algo.replace('_', ' ').title(), 
         'Time (seconds)': time_taken,
         'Time (minutes)': time_taken / 60 if time_taken else np.nan}
        for algo, time_taken in detection_results['timing'].items()
        if time_taken is not None
    ])
    timing_df = timing_df.sort_values('Time (seconds)', ascending=False)
    
    print("Execution Time Comparison")
    print("=" * 60)
    print(timing_df.to_string(index=False))
    print("=" * 60)
    print(f"\nTotal computation time: {sum([t for t in detection_results['timing'].values() if t is not None]):.2f} seconds")
else:
    print("⚠️ Timing information not available")


## 5. Community Size Statistics

Display statistics about community sizes for each detection method.


In [ ]:
# Display community size statistics for each method
print("Community Size Statistics by Algorithm")
print("=" * 80)

for method_name in ['louvain', 'label_propagation', 'greedy_modularity']:
    if method_name in detection_results and detection_results[method_name] is not None:
        communities, modularity = detection_results[method_name]
        sizes = get_community_sizes(communities)
        
        print(f"\n{method_name.replace('_', ' ').title()}:")
        print(f"  Number of communities: {len(communities):,}")
        print(f"  Modularity: {modularity:.6f}")
        print(f"  Size statistics:")
        print(f"    Min: {sizes.min()}")
        print(f"    Max: {sizes.max()}")
        print(f"    Mean: {sizes.mean():.2f}")
        print(f"    Median: {sizes.median():.2f}")
        print(f"    Std: {sizes.std():.2f}")

print("\n" + "=" * 80)


## 6. Evaluate Against Ground Truth

Evaluate all detection methods against ground-truth communities using NMI and ARI metrics.


In [ ]:
# Evaluate all methods against ground truth
print("Evaluating all methods against ground truth...")
print("=" * 60)

evaluation_results = compare_all_methods(detection_results, ground_truth_dict, G)

print("\nEvaluation Results Comparison:")
print("=" * 80)
print(evaluation_results.to_string(index=False))
print("=" * 80)


In [ ]:
# Prepare communities dictionary for visualization
os.makedirs("results/figures", exist_ok=True)

communities_dict = {}
for method in ['louvain', 'label_propagation', 'greedy_modularity']:
    if method in detection_results and detection_results[method] is not None:
        communities_dict[method] = detection_results[method][0]

# Add ground truth (convert dict to list of sets)
gt_communities_dict = {}
for node, comm_id in ground_truth_dict.items():
    if comm_id not in gt_communities_dict:
        gt_communities_dict[comm_id] = set()
    gt_communities_dict[comm_id].add(node)
communities_dict['ground_truth'] = list(gt_communities_dict.values())

# Plot size distribution
print("Creating community size distribution plot...")
plot_community_size_distribution(
    communities_dict,
    save_path="results/figures/community_size_distribution.png"
)
print("✅ Saved: results/figures/community_size_distribution.png")


### 7.2 Modularity Comparison

Compare modularity scores across all algorithms.


### 7.2 Modularity Comparison

Compare modularity scores across all algorithms.


In [ ]:
# Plot modularity comparison
print("Creating modularity comparison plot...")
plot_modularity_comparison(
    detection_results,
    save_path="results/figures/community_modularity_comparison.png"
)
print("✅ Saved: results/figures/community_modularity_comparison.png")


### 7.3 NMI and ARI Comparison

Compare evaluation metrics (NMI and ARI) across all algorithms.


In [ ]:
# Plot evaluation metrics
if not evaluation_results.empty:
    print("Creating evaluation metrics comparison plot...")
    plot_evaluation_metrics(
        evaluation_results,
        save_path="results/figures/community_evaluation_metrics.png"
    )
    print("✅ Saved: results/figures/community_evaluation_metrics.png")
else:
    print("⚠️ No evaluation results available for plotting")


### 7.4 Visualization of Largest Communities

Visualize the largest communities from the best-performing algorithm.


In [ ]:
# Find best method by modularity
best_method = None
best_modularity = -1
for method in ['louvain', 'label_propagation', 'greedy_modularity']:
    if method in detection_results and detection_results[method] is not None:
        _, mod = detection_results[method]
        if mod > best_modularity:
            best_modularity = mod
            best_method = method

if best_method and detection_results[best_method] is not None:
    communities, _ = detection_results[best_method]
    print(f"Visualizing largest communities from {best_method.replace('_', ' ').title()}...")
    print(f"   (Modularity: {best_modularity:.6f})")
    
    plot_largest_communities(
        G,
        communities,
        k=5,
        save_path="results/figures/community_largest_visualization.png",
        max_nodes_per_community=200  # Sample for large communities
    )
    print("✅ Saved: results/figures/community_largest_visualization.png")
else:
    print("⚠️ No valid communities found for visualization")


In [ ]:
# Analyze overlap for best method
if best_method and detection_results[best_method] is not None:
    communities, _ = detection_results[best_method]
    
    print(f"Analyzing community overlap for {best_method.replace('_', ' ').title()}...")
    print("=" * 60)
    
    overlap_results = analyze_community_overlap(communities, ground_truth_dict)
    
    print(f"Best matches found: {overlap_results['overlap_statistics']['num_matches']}")
    print(f"Average match score: {overlap_results['overlap_statistics']['avg_match_score']:.4f}")
    print(f"Max match score: {overlap_results['overlap_statistics']['max_match_score']:.4f}")
    print(f"Unmatched detected communities: {overlap_results['overlap_statistics']['num_unmatched_detected']}")
    print(f"Unmatched ground-truth communities: {overlap_results['overlap_statistics']['num_unmatched_ground_truth']}")
    print("=" * 60)


### Key Findings and Insights:

#### **Algorithm Performance Comparison:**

1. **Modularity Scores:**
   - Higher modularity indicates better community structure
   - Modularity > 0.3 is generally considered good
   - Compare which algorithm achieves highest modularity

2. **NMI (Normalized Mutual Information):**
   - Range: [0, 1] where 1 = perfect agreement with ground truth
   - Measures how well detected communities match ground truth
   - Higher NMI = better alignment with known communities

3. **ARI (Adjusted Rand Index):**
   - Range: [-1, 1] where 1 = perfect agreement
   - Adjusted for chance, more robust than raw Rand Index
   - Higher ARI = better agreement with ground truth

4. **Execution Time:**
   - Louvain: Typically fastest, good for large graphs
   - Label Propagation: Very fast, but may find fewer communities
   - Greedy Modularity: Slower, but often finds good communities

#### **Which Algorithm Performs Best?**

Based on the results above:

- **Best Modularity:** Algorithm with highest modularity score
- **Best NMI:** Algorithm with highest NMI (best match to ground truth)
- **Best ARI:** Algorithm with highest ARI (best agreement with ground truth)
- **Best Overall:** Consider all metrics together

#### **Interpretation:**

- **Louvain Algorithm:**
  - Fast and efficient for large networks
  - Good balance between speed and quality
  - Often finds communities with high modularity

- **Label Propagation:**
  - Very fast, suitable for very large graphs
  - May find fewer, larger communities
  - Good for networks with clear community structure

- **Greedy Modularity:**
  - Slower but thorough
  - Maximizes modularity directly
  - Good for smaller to medium-sized networks

#### **Business Implications:**

For the Amazon co-purchasing network:
- Communities represent product categories or purchasing patterns
- High-quality communities can inform:
  - Product recommendations
  - Marketing segmentation
  - Inventory management
  - Cross-selling strategies
- Understanding community structure helps identify:
  - Product clusters
  - Customer behavior patterns
  - Market segments


In [ ]:
# Create output directory
output_dir = "data/results/communities"
os.makedirs(output_dir, exist_ok=True)
os.makedirs("results/tables", exist_ok=True)

print(f"Saving all results to {output_dir}/...")

# Save detection results summary
summary_data = []
for method_name in ['louvain', 'label_propagation', 'greedy_modularity']:
    if method_name in detection_results and detection_results[method_name] is not None:
        communities, modularity = detection_results[method_name]
        sizes = get_community_sizes(communities)
        summary_data.append({
            'Method': method_name.replace('_', ' ').title(),
            'Num_Communities': len(communities),
            'Modularity': modularity,
            'Avg_Size': sizes.mean(),
            'Min_Size': sizes.min(),
            'Max_Size': sizes.max(),
            'Median_Size': sizes.median(),
            'Runtime_Seconds': detection_results['timing'].get(method_name, np.nan)
        })

summary_df = pd.DataFrame(summary_data)
summary_df.to_csv(os.path.join(output_dir, "detection_summary.csv"), index=False)
print(f"✅ Detection summary saved: {output_dir}/detection_summary.csv")

# Save evaluation results
if not evaluation_results.empty:
    evaluation_results.to_csv(os.path.join(output_dir, "evaluation_comparison.csv"), index=False)
    print(f"✅ Evaluation comparison saved: {output_dir}/evaluation_comparison.csv")

# Save timing information
timing_df = pd.DataFrame([
    {'Algorithm': algo.replace('_', ' ').title(), 'Time_Seconds': time_taken}
    for algo, time_taken in detection_results['timing'].items()
    if time_taken is not None
])
timing_df.to_csv(os.path.join(output_dir, "timing_comparison.csv"), index=False)
print(f"✅ Timing comparison saved: {output_dir}/timing_comparison.csv")

# Create comprehensive HTML report
print("\nCreating comprehensive HTML report...")
create_community_report(
    detection_results,
    evaluation_results,
    output_path="results/tables/community_report.html"
)
print("✅ HTML report created: results/tables/community_report.html")

print(f"\n✅ All results saved successfully!")
print(f"   Output directory: {output_dir}/")
print(f"   Figures: results/figures/")
print(f"   Reports: results/tables/")


## Summary

This notebook has completed comprehensive community detection analysis:

✅ **Graph Loaded**: Preprocessed Amazon co-purchasing network  
✅ **Ground Truth Loaded**: 271,570 communities for evaluation  
✅ **Algorithms Executed**: Louvain, Label Propagation, Greedy Modularity  
✅ **Performance Evaluated**: NMI, ARI, Modularity metrics computed  
✅ **Results Visualized**: Size distributions, comparisons, network visualizations  
✅ **Results Saved**: All outputs saved to data/results/communities/  

### Next Steps:
- Use detected communities for product recommendations
- Analyze community structure for business insights
- Compare with other community detection methods
- Explore community evolution over time (if temporal data available)
